In [1]:
import wrangle as w 
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_text
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = w.wrangle_df()

clean.csv not detected.
Acquiring and Preparing Data
Using cached CSV
clean.csv not detected.
processing capstone.csv
clean.csv ready for future use


In [3]:
df.columns

Index(['title', 'success', 'success_rating', 'genres', 'cast_actor_1',
       'cast_actor_2', 'cast_actor_3', 'total_n_cast', 'budget', 'revenue',
       'profit_amount', 'vote_average', 'vote_count', 'production_companies',
       'production_countries', 'overview', 'popularity', 'runtime',
       'profitable', 'release_date', 'release_year', 'imdb_id',
       'is_genre_adventure', 'is_genre_horror', 'is_genre_drama',
       'is_genre_scifi', 'is_genre_romance', 'is_genre_thriller',
       'is_genre_crime', 'is_genre_comedy', 'is_genre_animation',
       'is_genre_action', 'is_genre_mystery', 'is_genre_fantasy',
       'is_genre_documentary'],
      dtype='object')

In [5]:
df

,title,success,success_rating,genres,cast_actor_1,cast_actor_2,cast_actor_3,total_n_cast,budget,revenue,...,is_genre_scifi,is_genre_romance,is_genre_thriller,is_genre_crime,is_genre_comedy,is_genre_animation,is_genre_action,is_genre_mystery,is_genre_fantasy,is_genre_documentary
id,,,,,,,,,,,,,,,,,,,,,
5,Four Rooms,0,3.493750,Crime Comedy,Tim Roth,Antonio Banderas,Jennifer Beals,24.0,4000000.0,4300000.0,...,False,False,False,True,True,False,False,False,False,False
6,Judgment Night,0,3.883820,Action Thriller Crime,Emilio Estevez,Cuba Gooding Jr.,Denis Leary,15.0,10000000.0,12136938.0,...,False,False,True,True,False,False,True,False,False,False
11,Star Wars,1,285.487448,Adventure Action Science Fiction,Mark Hamill,Harrison Ford,Carrie Fisher,106.0,11000000.0,775398007.0,...,True,False,False,False,False,False,True,False,False,False
12,Finding Nemo,1,38.013564,Animation Family,Albert Brooks,Ellen DeGeneres,Alexander Gould,24.0,94000000.0,940335536.0,...,False,False,False,False,False,True,False,False,False,False
13,Forrest Gump,1,50.537748,Comedy Drama Romance,Tom Hanks,Robin Wright,Gary Sinise,66.0,55000000.0,677945399.0,...,False,True,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433356,The Ottoman Lieutenant,0,0.019579,Romance Drama War,Michiel Huisman,Hera Hilmar,Josh Hartnett,24.0,40000000.0,240978.0,...,False,True,False,False,False,False,False,False,False,False
434119,Confidential Assignment,1,20.411972,Action Drama Comedy,Hyun Bin,Yoo Hae-jin,Kim Joo-hyuk,7.0,8520000.0,56100000.0,...,False,False,False,False,True,False,True,False,False,False
437031,Rescue Under Fire,0,1.525000,War,Ariadna Gil,Roberto Álamo,Antonio Garrido,13.0,10000000.0,5000000.0,...,False,False,False,False,False,False,False,False,False,False


In [27]:
excess = df[df['success_rating'] >= 10]

In [28]:
excess.shape

(1916, 22)

In [29]:
df.shape

(7282, 22)

In [30]:
1916/7282

0.26311452897555615

In [32]:
excess['success_rating'].sample(60)

id
18402      18.130332
923        25.437500
5780       13.866667
32275      10.540000
9036       10.088189
105        80.233634
19114      10.153961
19997      22.780000
10414      16.240748
124905     10.085513
11862      27.281237
1597       19.826643
388        14.954963
14412      10.676109
874        27.259615
16804      33.654961
8408       33.514286
7300       37.183141
381284     35.989011
9569       13.274800
869        10.141929
72570      22.880033
299245     13.065976
9470       18.164600
329865     15.578670
31357      15.526817
297761     12.568687
85038      16.333333
251        79.193182
1687       15.559620
10693      76.479070
12919      58.855467
7942       10.230000
11449      33.492400
954        19.166035
9816       22.059275
435        13.497956
16620      28.358833
40807      17.144655
114       115.750000
10222      30.863710
97630      11.123735
76338      12.891428
3981       16.300582
6844       35.394231
2928       41.153132
156022     12.414075
9062      

In [50]:
df['success_rating4'] = (df['revenue']/(df['budget'] * 1.2)) * (df['vote_average']/10)

In [ ]:
for index, row in df.iterrows():
        try:
            budget = df.at[index, 'budget']
            revenue = df.at[index, 'revenue']
            if (revenue >= budget * 2):
                profitable = True
            else:
                profitable = False
        except:     # if budget or revenue is empty
            profitable = np.nan

In [37]:
df['success'] = df['success_rating'] > 6.5

In [41]:
df.success_rating2.max()

6.673333333333334

In [43]:
df.success_rating3.min()

0.0

In [44]:
df.success_rating3.max()

6.673333333333334

In [45]:
df.profitable.value_counts()

False    4351
True     2931
Name: profitable, dtype: int64

In [51]:
df.success_rating4.value_counts()

0.000000     22
1.200000      4
0.800000      4
0.500000      3
0.155556      2
             ..
4.417106      1
18.643269     1
0.009862      1
4.620417      1
0.610714      1
Name: success_rating4, Length: 7217, dtype: int64

In [52]:
df.success_rating.describe()

count    7282.000000
mean        9.566970
std        21.569372
min         0.000000
25%         0.875969
50%         4.133958
75%        10.483487
max       667.333333
Name: success_rating, dtype: float64

In [54]:
success.value_counts()

False    4482
True     2800
Name: success, dtype: int64

In [60]:
df['success'] = df['success']*1

In [62]:
df['profitable'] = df['profitable']*1

In [63]:
df.profitable.value_counts()

0    4351
1    2931
Name: profitable, dtype: int64

In [6]:
df['is_genre_adventure2'] = df.genres.apply(lambda genre_list: 'Adventure' in genre_list)*1

In [7]:
df

,title,success,success_rating,genres,cast_actor_1,cast_actor_2,cast_actor_3,total_n_cast,budget,revenue,...,is_genre_romance,is_genre_thriller,is_genre_crime,is_genre_comedy,is_genre_animation,is_genre_action,is_genre_mystery,is_genre_fantasy,is_genre_documentary,is_genre_adventure2
id,,,,,,,,,,,,,,,,,,,,,
5,Four Rooms,0,3.493750,Crime Comedy,Tim Roth,Antonio Banderas,Jennifer Beals,24.0,4000000.0,4300000.0,...,False,False,True,True,False,False,False,False,False,0
6,Judgment Night,0,3.883820,Action Thriller Crime,Emilio Estevez,Cuba Gooding Jr.,Denis Leary,15.0,10000000.0,12136938.0,...,False,True,True,False,False,True,False,False,False,0
11,Star Wars,1,285.487448,Adventure Action Science Fiction,Mark Hamill,Harrison Ford,Carrie Fisher,106.0,11000000.0,775398007.0,...,False,False,False,False,False,True,False,False,False,1
12,Finding Nemo,1,38.013564,Animation Family,Albert Brooks,Ellen DeGeneres,Alexander Gould,24.0,94000000.0,940335536.0,...,False,False,False,False,True,False,False,False,False,0
13,Forrest Gump,1,50.537748,Comedy Drama Romance,Tom Hanks,Robin Wright,Gary Sinise,66.0,55000000.0,677945399.0,...,True,False,False,True,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433356,The Ottoman Lieutenant,0,0.019579,Romance Drama War,Michiel Huisman,Hera Hilmar,Josh Hartnett,24.0,40000000.0,240978.0,...,True,False,False,False,False,False,False,False,False,0
434119,Confidential Assignment,1,20.411972,Action Drama Comedy,Hyun Bin,Yoo Hae-jin,Kim Joo-hyuk,7.0,8520000.0,56100000.0,...,False,False,False,True,False,True,False,False,False,0
437031,Rescue Under Fire,0,1.525000,War,Ariadna Gil,Roberto Álamo,Antonio Garrido,13.0,10000000.0,5000000.0,...,False,False,False,False,False,False,False,False,False,0


In [10]:
df['is_genre_comedy2'] = df.genres.apply(lambda genre_list: 'Comedy' in genre_list)*1

In [11]:
df

,title,success,success_rating,genres,cast_actor_1,cast_actor_2,cast_actor_3,total_n_cast,budget,revenue,...,is_genre_thriller,is_genre_crime,is_genre_comedy,is_genre_animation,is_genre_action,is_genre_mystery,is_genre_fantasy,is_genre_documentary,is_genre_adventure2,is_genre_comedy2
id,,,,,,,,,,,,,,,,,,,,,
5,Four Rooms,0,3.493750,Crime Comedy,Tim Roth,Antonio Banderas,Jennifer Beals,24.0,4000000.0,4300000.0,...,False,True,True,False,False,False,False,False,0,1
6,Judgment Night,0,3.883820,Action Thriller Crime,Emilio Estevez,Cuba Gooding Jr.,Denis Leary,15.0,10000000.0,12136938.0,...,True,True,False,False,True,False,False,False,0,0
11,Star Wars,1,285.487448,Adventure Action Science Fiction,Mark Hamill,Harrison Ford,Carrie Fisher,106.0,11000000.0,775398007.0,...,False,False,False,False,True,False,False,False,1,0
12,Finding Nemo,1,38.013564,Animation Family,Albert Brooks,Ellen DeGeneres,Alexander Gould,24.0,94000000.0,940335536.0,...,False,False,False,True,False,False,False,False,0,0
13,Forrest Gump,1,50.537748,Comedy Drama Romance,Tom Hanks,Robin Wright,Gary Sinise,66.0,55000000.0,677945399.0,...,False,False,True,False,False,False,False,False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433356,The Ottoman Lieutenant,0,0.019579,Romance Drama War,Michiel Huisman,Hera Hilmar,Josh Hartnett,24.0,40000000.0,240978.0,...,False,False,False,False,False,False,False,False,0,0
434119,Confidential Assignment,1,20.411972,Action Drama Comedy,Hyun Bin,Yoo Hae-jin,Kim Joo-hyuk,7.0,8520000.0,56100000.0,...,False,False,True,False,True,False,False,False,0,1
437031,Rescue Under Fire,0,1.525000,War,Ariadna Gil,Roberto Álamo,Antonio Garrido,13.0,10000000.0,5000000.0,...,False,False,False,False,False,False,False,False,0,0
